In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import bar_chart_race as bcr
import matplotlib as mpl
import os
from openpyxl import load_workbook
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

principal = os.getcwd()

os.chdir(principal)

mega = pd.read_excel('dados_update.xlsx')

quadro = mega.drop(['Concurso', 'Data'], axis=1)

print(quadro.count())


bola 1    2908
bola 2    2908
bola 3    2908
bola 4    2908
bola 5    2908
bola 6    2908
dtype: int64


# Refazer essa tabela com dados congruentes

In [ ]:
contagem = {}

for numero in range(1, 61):
    contagem_numero = {}
    for coluna in quadro.columns:
        temp = 0
        for instancia in quadro[coluna]:
            if instancia == numero:
                temp += 1
        
        contagem_numero[coluna] = temp
    
    contagem[numero] = contagem_numero

tabela = pd.DataFrame(contagem).T

tabela.index.name = 'Número'

tabela['Total'] = tabela.sum(axis=1)

tabela['Média das Frequencias'] = int((len(mega) * 6)/60)

# O calculo do descio padrão esta errado, pois teria que ser por bola de cada número, não da frequencias de cada bola
# tabela['Desvio Padrão'] = round(tabela['Total'].std(), 1)

tabela['Média por bola'] = round(tabela['Total'] / len(quadro.columns), 2)

tabela['Média por bola Esperada'] = int((1/60)*len(mega))

tabela['Desvio Padrão das Bolas'] = round(tabela['Média por bola'].std(), 1)

tabela['Probabilidade (E)'] = round(tabela['Total'] / (len(quadro)), 2)

save = os.path.join(principal, 'Metricas')

os.makedirs(save, exist_ok=True)

os.chdir(save)

tabela.to_excel('Mega_Sena_Métricas.xlsx', sheet_name='Métricas')

os.chdir(principal)

display(tabela)



# O Código abaixo gera um gráfico em barra de acordo com a frequencia de cada número, dando um comparativo (contraste) nos números com maior, menor e média frequência

In [ ]:
total_de_Frequencia = tabela['Total']

tabela2 = tabela.reset_index()

caminho_metricas = os.path.join(principal, 'Metricas')

os.chdir(caminho_metricas)

media = total_de_Frequencia.mean() 
maxima = total_de_Frequencia.max()
minima = total_de_Frequencia.min()

# ========================================================================================================
abaixo = tabela[tabela['Total'] < tabela['Média das Frequencias']]

with pd.ExcelWriter('./Mega_Sena_Métricas.xlsx', mode='a', engine='openpyxl') as writer:
    abaixo.to_excel(writer, sheet_name="Abaixo da média", index=False)

maior = tabela[tabela['Total'] > tabela['Média das Frequencias']]

with pd.ExcelWriter('./Mega_Sena_Métricas.xlsx', mode='a', engine='openpyxl') as writer:
    maior.to_excel(writer, sheet_name="Acima da média", index=False)

igual = tabela[tabela['Total'] == tabela['Média das Frequencias']]

with pd.ExcelWriter('./Mega_Sena_Métricas.xlsx', mode='a', engine='openpyxl') as writer:
    igual.to_excel(writer, sheet_name="Igual da média", index=False)
# ========================================================================================================

abaixo_da_media = len(tabela[tabela['Total'] < tabela['Média das Frequencias']])

maior_da_media = len(tabela[tabela['Total'] > tabela['Média das Frequencias']])

igual_a_media = len(tabela[tabela['Total'] == tabela['Média das Frequencias']])


plt.figure(figsize=(15, 8))

plt.bar(tabela2['Número'], total_de_Frequencia.values)
plt.axhline(y=media, color='red', linestyle='--', label=f'Média: {media:.2f} \n Abaixo:{abaixo_da_media} \n Igual:{igual_a_media}\n Acima:{maior_da_media}')
plt.axhline(y=maxima, color='yellow', linestyle='--', label=f'Maxima: {maxima:.2f}')
plt.axhline(y=minima, color='blue', linestyle='--', label=f'Minima: {minima:.2f}')

plt.title('Frequências com Linha da Média')
plt.xlabel('Número')
plt.ylabel('Total de Frequência')

plt.legend()  
plt.savefig('FrequenciasTotal')

plt.tight_layout()
plt.show()

os.chdir(principal)

In [ ]:
v = 9
while (v != 0):
    v = int(input("Digite o número que deseja analisar ou 0 para sair"))

    if (v > 0 & v < 60):
        linha = tabela.loc[v]

        linha_filtrada = linha[["bola 1", "bola 2", "bola 3","bola 4", "bola 5", "bola 6"]]

        media = linha_filtrada.mean()
        maxima = linha_filtrada.max().max()
        minima = linha_filtrada.min().min()

        plt.figure(figsize=(15,6))
        plt.axhline(y=media, color='red', linestyle='--', label=f'Média: {media:.2f}')
        plt.axhline(y=maxima, color='yellow', linestyle='--', label=f'Maxima: {maxima:.2f}')
        plt.axhline(y=minima, color='blue', linestyle='--', label=f'Minima: {minima:.2f}')
        plt.bar(linha_filtrada.index, linha_filtrada.values)

        plt.title(f"Frequência por bola do número {v}")
        plt.xlabel('Colunas')
        plt.ylabel('Frequência')

        plt.legend()

        plt.tight_layout()
    
        diretorio_frequencia_bola = os.path.join(principal,'FrequenciaBolas')

        os.makedirs(diretorio_frequencia_bola, exist_ok=True)

        os.chdir(diretorio_frequencia_bola)
        plt.savefig(f"FrequenciaPorBolaDo{v}")
        plt.show()
    elif (v == 0):
        print("Obrigado por me usar! Tchau")
    else:
        print('Número inválido, tente novamente!')

## Analise de Frequencia de Cada Bola

O codigo abaixo gera um gráfico e uma tabela para as frequencias de cada bola de todos os números.

In [ ]:
for bola in range(1,7):

    os.chdir(principal)

    dados = mega[f'bola {bola}']

    dicio = {i: 0 for i in range(1, 61)}

    lista_dados = []

    for i in range(len(dados) - 1, -1, -1):
        valor = int(dados[i])
        dicio[valor] += 1
        lista_dados.append(dicio.copy())

    historico_bola = pd.DataFrame(lista_dados)

    diretorioAtual = os.getcwd()

    caminho = os.path.join(diretorioAtual, f'Bola {bola}')

    os.makedirs(caminho, exist_ok=True)

    os.chdir(caminho)

    historico_bola.to_excel(f'Frequencias_Historicas_Mega_sena_bola_{bola}.xlsx')

    mpl.rcParams['animation.ffmpeg_path'] = r'C:\ffmpeg\bin\ffmpeg.exe' 
    mpl.rcParams['animation.embed_limit'] = 1000

    recentes = historico_bola.tail(50)

    bcr.bar_chart_race(
        df=recentes,
        filename=f'frequenciaMegaSena{bola}.mp4',
        steps_per_period=10,
        orientation='v',
        figsize=(16,10),
        interpolate_period=True,
        title="Frequencias dos números desde 1996",
        cmap='dark24',
        perpendicular_bar_func='mean', 
    )



In [ ]:
# variancia = pd.DataFrame([dicio.copy()])

# print(variancia)

# display(tabela)

# Faz uma cópia segura do dataframe
variancia = tabela[['Total', 'Total Esperado']].copy()

# # Cria a coluna Diferença ao quadrado
# variancia["Diferença"] = (variancia['Total'] - variancia['Total Esperado']) 

# variancia['Diferença²'] = variancia['Diferença'] **2

# variancia['Média da Diferença²'] = int(variancia['Diferença²'].mean())

# Cálculo de variância e desvio padrão para 'Total'
variancia_total = round(variancia['Total'].var())
desvio_padrao = round(variancia['Total'].std())


# Impressão dos resultados
print('Variância:', variancia_total)
print('Desvio padrão:', desvio_padrao)

display(variancia)


# O código abaixo gera todos os concursos onde o número com a maior frequencia apareceu

Caso queiramos, conseguimos definir qual número gostariamos de pesquisar e a quantidade de resultados retornado.

In [13]:
numero_mais_frequente = tabela['Total'].idxmax()
print("Número mais frequente:", numero_mais_frequente)

# Ver todas as linhas onde esse número aparece em qualquer uma das bolas
apareceu_em = mega[
    (mega['bola 1'] == numero_mais_frequente) |
    (mega['bola 2'] == numero_mais_frequente) |
    (mega['bola 3'] == numero_mais_frequente) |
    (mega['bola 4'] == numero_mais_frequente) |
    (mega['bola 5'] == numero_mais_frequente) |
    (mega['bola 6'] == numero_mais_frequente)
]

display(apareceu_em.head(10))

Número mais frequente: 10


,Concurso,Data,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
9,2899,09/08/2025,28,44,22,42,10,51
15,2893,26/07/2025,10,50,41,45,40,48
23,2885,08/07/2025,56,25,10,28,36,37
39,2869,29/05/2025,2,53,13,41,10,40
40,2868,27/05/2025,10,38,26,22,51,19
52,2856,26/04/2025,10,27,48,3,5,38
63,2845,27/03/2025,40,31,56,52,10,54
72,2836,06/03/2025,30,6,50,1,42,10
82,2826,08/02/2025,20,58,54,10,60,6
90,2818,21/01/2025,7,15,25,47,12,10
